In [1]:
import numpy as np
import matplotlib.pyplot as plt

1*. Измените функцию calc_logloss так, чтобы нули по возможности не попадали в np.log.

In [3]:
def calc_logloss(y, y_pred):
    err = - np.mean(y * np.log(y_pred  + 1e-7) + (1.0 - y) * np.log(1.0 - y_pred + 1e-7))
    err = np.sum(err)
    return err

2. Подберите аргументы функции eval_model для логистической регрессии таким образом, чтобы log loss был минимальным.

In [4]:
X = np.array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
              [1, 1, 2, 5, 3, 0, 5, 10, 1, 2],
              [500, 700, 750, 600, 1450, 800, 1500, 2000, 450, 1000],
              [1, 1, 2, 1, 2,  1, 3, 3, 1, 2]], dtype = np.float64)
y = np.array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1], dtype = np.float64)

In [5]:
def calc_std_feat(x):
    res = (x - x.mean()) / x.std()
    return res

In [6]:
X_st = X.copy()
X_st[2, :] = calc_std_feat(X[2, :])

In [7]:
def sigmoid(z):
    res = 1 / (1 + np.exp(-z))
    return res

In [8]:
def eval_model(X, y, iterations, alpha=1e-4):
    np.random.seed(42)
    W = np.random.randn(X.shape[0])
    n = X.shape[1]
    for i in range(1, iterations+1):
        z = np.dot(W, X)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred)
        W -= alpha * (1/n * np.dot((y_pred - y), X.T))
        if i % (iterations / 10) == 0:
            print(i, W, err)
    return W

In [48]:
W = eval_model(X_st, y, iterations=100000, alpha=1e-1)

10000 [-10.744631    -1.38398009  -2.42084781   9.13416686] 0.23214405251334216
20000 [-15.4271433   -1.7813775   -3.82869298  12.9172959 ] 0.19343479830681998
30000 [-19.03182249  -2.10015312  -4.88452655  15.84696481] 0.17055122530831654
40000 [-21.99871339  -2.36615096  -5.74176305  18.25901563] 0.155085238722678
50000 [-24.52654832  -2.59423698  -6.46573557  20.31274988] 0.14388088912953587
60000 [-26.73396956  -2.7939693   -7.09418556  22.10438247] 0.13535199591567162
70000 [-28.69836096  -2.97180995  -7.65118589  23.69690993] 0.128608347174879
80000 [-30.47284279  -3.13229609  -8.15298507  25.13364996] 0.12311350887868482
90000 [-32.09529764  -3.27871593  -8.61101651  26.4455232 ] 0.11852609972432315
100000 [-33.59359124  -3.41351803  -9.03359875  27.65528749] 0.11461902677322375


3. Создайте функцию calc_pred_proba, возвращающую предсказанную вероятность класса 1 (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred_proba).

In [49]:
def calc_pred_proba(W, X):
    z = np.dot(W, X)
    pred_proba = sigmoid(z)
    return pred_proba

In [50]:
pred_proba = calc_pred_proba(W, X_st)
pred_proba

array([3.76929484e-01, 1.43636180e-02, 9.99999995e-01, 1.10343509e-07,
       9.32592337e-01, 6.42801817e-02, 1.00000000e+00, 2.06617132e-02,
       6.05609842e-01, 9.99999456e-01])

4. Создайте функцию calc_pred, возвращающую предсказанный класс (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred).

In [51]:
def calc_pred(W, X, threshold = 0.5):
    z = np.dot(W, X)
    pred_proba = sigmoid(z)
    y_pred = np.where(pred_proba > threshold, 1, 0)
    return y_pred

In [52]:
y_pred = calc_pred(W, X_st, 0.5)
y_pred

array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1])

5. Посчитайте Accuracy, матрицу ошибок, точность и полноту, а также F1 score.

In [53]:
acc = np.mean(y == y_pred)
print('Accuracy: ', acc)

Accuracy:  1.0


In [54]:
def compute_confusion_matrix(true, pred):
    K = len(np.unique(true)) 
    result = np.zeros((K, K))
    
    for i in range(len(true)):
        result[true[i]][pred[i]] += 1

    return result

In [61]:
y = y.astype(int)

In [63]:
conf_mx = compute_confusion_matrix(y, y_pred)
conf_mx

array([[5., 0.],
       [0., 5.]])

In [68]:
prec = conf_mx[0][0] / (conf_mx[0][0] + conf_mx[0][1])
prec

1.0

In [69]:
rec = conf_mx[0][0] / (conf_mx[0][0] + conf_mx[1][0])
rec

1.0

In [70]:
f1 = 2 * prec * rec / (prec + rec)
f1

1.0

6. Могла ли модель переобучиться? Почему?

*Да, потому что в модели мало входных данных*

7*. Создайте функции eval_model_l1 и eval_model_l2 с применением L1 и L2 регуляризаций соответственно.

In [71]:
def eval_model_l1(X, y, iterations, alpha=1e-4, lambda_ = 1e-7):
    np.random.seed(42)
    W = np.random.randn(X.shape[0])
    n = X.shape[1]
    for i in range(1, iterations+1):
        z = np.dot(W, X)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred)
        W -= alpha * (1/n * np.dot((y_pred - y), X.T) + lambda_ * np.sign(W))
        if i % (iterations / 10) == 0:
            print(i, W, err)
    return W

In [72]:
def eval_model_l1(X, y, iterations, alpha=1e-4, lambda_ = 1e-7):
    np.random.seed(42)
    W = np.random.randn(X.shape[0])
    n = X.shape[1]
    for i in range(1, iterations+1):
        z = np.dot(W, X)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred)
        W -= alpha * (1/n * np.dot((y_pred - y), X.T) + lambda_ * W)
        if i % (iterations / 10) == 0:
            print(i, W, err)
    return W